In [1]:
import requests
import pickle
import json
import os
import googlemaps
from datetime import datetime
import sqlite3
import pandas as pd
from ast import literal_eval


In [2]:
class GetRawAPI():
    
    def __init__(self):
        print("Let's get started")
        pass

    @staticmethod 
    def create_param_sets():
        now = datetime.now()
        driving_parameters = {'origin' : "120 Benchley Pl, Bronx NY"
                             , 'destination' : "809 Washington Street"
                             , 'mode' : "driving"
                             , 'departure_time' : now}

        transit_parameters = {'origin' : "120 Benchley Pl, Bronx NY"
                             , 'destination' : "809 Washington Street"
                             , 'mode' : "transit"
                             , 'departure_time' : now}
        param_set_list = [driving_parameters, transit_parameters]
        return(param_set_list)

    @staticmethod
    def get_gmap_directions(param_set_list, 
                            google_key_path = '/home/malcolm/credentials/google_transit_api.pkl'):
        # Start google client 
        with open(google_key_path, 'rb') as hnd:
            api_key = pickle.load(hnd)
        gmaps = googlemaps.Client(key=api_key)

        # Make API Calls 
        updated_params_list = []
        for param_set in param_set_list:
            param_set2 = param_set.copy()
            temp_result = gmaps.directions(**param_set)

            date = str(param_set2['departure_time'].date())
            time = str(param_set2['departure_time'].strftime('%H:%M'))
            param_set2.update({'Raw Response' : temp_result
                              , 'Date' : date
                              , 'Time' : time})
            updated_params_list.append(param_set2)

        # Fix up dataframe 
        updated_params_df = pd.DataFrame(updated_params_list)
        updated_params_df['Raw Response'] = updated_params_df['Raw Response'].astype(str)
        updated_params_df['departure_time'] = updated_params_df['departure_time'].astype(str)
        return(updated_params_df)

    @staticmethod
    def update_db(updated_params_df,     
                  db_path = '/home/malcolm/WorkTransit/data/',
                  db_file_name = 'transit.db'):
        # Ensure paths and files are ready 
        os.makedirs(db_path, exist_ok=True)
        if db_path[-1] != '/':
            db_path = db_path + '/'
        db_loc = db_path + db_file_name
        
        # Create SQL Connection and append to db
        con = sqlite3.connect(db_loc)
        updated_params_df.to_sql('Raw_Responses', con, index=False, if_exists='append' )
        
        # Close connection
        con.commit()
        con.close()
        print(f"Added to {db_loc}")
    
    def execute(self):
        self.param_set_list = self.create_param_sets()
        self.updated_params_df = self.get_gmap_directions(self.param_set_list)
        self.update_db(self.updated_params_df)

In [3]:
db_updater = GetRawAPI()
db_updater.execute()

print(db_updater.updated_params_df)

Let's get started
Added to /home/malcolm/WorkTransit/data/transit.db
                      origin            destination     mode  \
0  120 Benchley Pl, Bronx NY  809 Washington Street  driving   
1  120 Benchley Pl, Bronx NY  809 Washington Street  transit   

               departure_time  \
0  2022-08-16 20:35:08.420533   
1  2022-08-16 20:35:08.420533   

                                        Raw Response        Date   Time  
0  [{'bounds': {'northeast': {'lat': 40.8745759, ...  2022-08-16  20:35  
1  [{'bounds': {'northeast': {'lat': 40.8798673, ...  2022-08-16  20:35  


/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [4]:
dir(db_updater)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'create_param_sets',
 'execute',
 'get_gmap_directions',
 'param_set_list',
 'update_db',
 'updated_params_df']